In [3]:
from abc import ABC, abstractmethod

class LinearCode(ABC):
    
    def __init__(self, code=None):
        self.code = code
    
    def set_code(self, code):
        self.code = code
    
    def get_code(self):
        return self.code
    
    @abstractmethod
    def get_parity_check_matrix(self):
        ...
                
    @staticmethod
    def serialize_instance(self, target_file_name):
        pass
    
    @staticmethod
    def load_from_file(self):
        pass

In [4]:
class GRSCode(LinearCode):
    
    def __init__(self, code=None, n=10, k=5, q=11):
        if code == None:
            self.code = self.generate_code(n, k, q)
        else:
            self.code = code
    
    def get_parity_check_matrix(self):
        return self.code.parity_check_matrix()
    
    def generate_code(self, n, k, q):
        F = GF(q)
        return codes.GeneralizedReedSolomonCode(F.list()[:n], k)

In [5]:
code = GRSCode()
print(code.get_parity_check_matrix())

[10  9  8  7  6  5  4  3  2  1]
[ 0  9  5 10  2  3  2 10  5  9]
[ 0  9 10  8  8  4  1  4  7  4]
[ 0  9  9  2 10  9  6  6  1  3]
[ 0  9  7  6  7  1  3  9  8  5]


In [6]:

class CodeBasedCryptosystem(ABC):
    def __init__(self, code, public_key=None, private_key=None):
        self.code = code
        if public_key == None or private_key == None:
            self.public_key, self.private_key = self.generate_keypair()
        else:
            self.public_key = public_key
            self.private_key = private_key
    
    @abstractmethod
    def generate_keypair(self):
        ...
    
    @abstractmethod
    def encrypt(self, message):
        ...
    
    @abstractmethod
    def decrypt(self, cryptotext):
        ...

In [7]:
from math import floor

class Niederreiter(CodeBasedCryptosystem):
    def generate_random_nonsingular_matrix(self, size):
        S = random_matrix(ZZ, size)
        
        while not S.is_singular() and S.determinant()==0:
            S = random_matrix(ZZ, size)
        
        return S
    
    def generate_random_permutation_matrix(self, n):
        return Permutations(n).random_element().to_matrix()
    
    def encode_(self, n, t):
        enc = [0]*n
        count = t
        while count>0:
            r = randrange(n)
            if enc[r] == 0:
                enc[r]+=1
                count-=1
        return matrix(enc)
    
    def encrypt(self, message=None):
        H_pub, t = self.public_key
        n = H_pub.ncols()
        
        if message == None:
            message = self.encode_(n, t)
        
#         print("Message is: "+ str(message))
        
        return matrix(QQ, H_pub) * matrix(QQ, message.transpose())
        
    
    def decrypt(self, cryptotext):
        H, S, P = self.private_key
        #print(cryptotext)
        print("SIZES \n")
        print((~S).nrows(), (~S).ncols())
        
        HPMt = (~S) * cryptotext
        return HPMt
        
    
    def generate_keypair(self):
        H = code.get_parity_check_matrix()
        n, k = H.ncols(), H.nrows()
        S = self.generate_random_nonsingular_matrix(n - k)
        P = self.generate_random_permutation_matrix(n)
        t = floor((n - k) / 2)
        
        return ((S * H * P, t), (H, S, P))

In [6]:
nied = Niederreiter(code=code)
print(nied.public_key)
print(nied.private_key)

([ 1  1  5  4  6  4  6  6  2  9]
[ 9  7  7  0  3  2  7  2  6  1]
[10  3  8  5  2 10  3  5  0  9]
[ 2  2  1  9  7  6  7  7  8  6]
[ 0  9  5  6  6 10  8  4  7  0], 2)
([10  9  8  7  6  5  4  3  2  1]
[ 0  9  5 10  2  3  2 10  5  9]
[ 0  9 10  8  8  4  1  4  7  4]
[ 0  9  9  2 10  9  6  6  1  3]
[ 0  9  7  6  7  1  3  9  8  5], [ -1 -10  -9 -38   1]
[  2  -6  -2   3   2]
[  1   4 -14  -6  -1]
[ -2   0   3  -3  -1]
[  0   0  -1  -4   0], [1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0])


In [17]:
enc = nied.encrypt()
print("Encrypted message is: \n"+str(enc))
dec = nied.decrypt(enc)
print("Decrypted message is: "+str(dec))

Message is: [0 0 0 1 0 0 0 1 0 0]
Encrypted message is: 
[ 5]
[16]
[11]
[ 4]
[11]
SIZES 

5 5
Decrypted message is: [   579/632]
[  -637/316]
[-3803/2528]
[-3069/2528]
[ -411/1264]


In [3]:
class GenericAttack(ABC):
    def __init__(self, H, syndrome, t):
        self.H = H
        self.syndrome = syndrome
        self.t = t
        
    @abstractmethod
    def attack(self):
        ...

In [4]:
class PrangeISD(GenericAttack):
    def innerLoop(self):
        pass
    def attack(self):
        pass

In [8]:
import unittest

class NiederreiterTestClass(unittest.TestCase):
       
    def test_key_generating(self):
        """ The method is implemented """
        code_Nied = Niederreiter(code=code)
        self.assertTrue(len(code_Nied.generate_keypair()[0]),2)
    def test_encrypt(self):
        """ The method is implemented """
        code_Nied = Niederreiter(code=code)
        self.assertTrue(len(code_Nied.encrypt()[0]),5)
        

In [9]:
import sys
suite = unittest.TestLoader().loadTestsFromTestCase(NiederreiterTestClass)
unittest.TextTestRunner(verbosity=2,stream=sys.stderr).run(suite)

test_encrypt (__main__.NiederreiterTestClass)
The method is implemented ... ok
test_key_generating (__main__.NiederreiterTestClass)
The method is implemented ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.337s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [2]:
import itertools
def lee_brickel(error_code,C,weight,p):
    G=C.generator_matrix()
    k,n=G.dimensions()
    Fstar=[1]
    while True:
        I = sample(range(n),k)
        g_I = G.matrix_from_columns(I)
        print('g_I',g_I)
        while not g_I.is_invertible():
            I = sample(range(n),k)
            g_I = G.matrix_from_columns(I)
            
        G = g_I.inverse() * G

        y_I_G = vector([error_code[i] for i in I]) * G

        y = error_code - y_I_G
        g = G.rows()

        for A in itertools.combinations(range(k), p):
            for m in itertools.product(Fstar, repeat=p):
                e = y - sum(m[i]*g[A[i]] for i in range(p))
                if e.hamming_weight() == weight:
                    return error_code - e
C = codes.random_linear_code(GF(2), 20,8 )
c = C.random_element()
print(c)
Chan = channels.StaticErrorRateChannel(C.ambient_space(), 5)
y = Chan(c)

nr_iter = 1
while lee_brickel(y, C, 5, 2)!=c:
    lee_brickel(y, C, 5, 2)
    nr_iter+= 1
print ('Number of iterations',nr_iter)

(1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1)
g_I [1 0 0 1 1 1 1 1]
[1 0 0 0 1 1 1 1]
[0 0 0 1 1 0 1 1]
[0 1 0 0 0 0 0 0]
[0 1 0 0 1 1 1 1]
[1 1 1 1 1 1 1 0]
[0 1 0 1 0 0 1 1]
[1 0 0 0 0 1 0 1]
g_I [1 0 1 1 1 1 1 1]
[1 1 1 1 1 1 1 0]
[0 0 1 0 0 0 1 1]
[0 1 0 0 0 0 1 0]
[1 0 1 0 1 0 0 0]
[1 1 1 1 1 1 1 1]
[0 0 1 0 0 0 1 0]
[1 0 0 0 1 1 0 0]
g_I [1 0 0 1 0 1 1 1]
[1 0 0 1 1 1 0 1]
[0 1 0 0 0 1 1 1]
[0 1 1 0 1 0 0 0]
[0 1 0 1 0 1 0 1]
[1 0 0 1 1 0 1 1]
[0 0 1 0 0 1 1 1]
[1 1 1 1 0 1 0 0]
g_I [1 1 1 0 1 0 1 1]
[0 1 0 0 1 1 1 1]
[1 1 1 0 0 0 1 1]
[0 0 0 1 0 0 0 0]
[0 1 0 1 1 0 1 1]
[1 1 1 1 1 1 0 1]
[1 0 0 1 0 0 1 1]
[0 0 0 0 1 0 1 0]
g_I [0 1 0 1 0 1 1 0]
[0 1 1 1 1 1 0 1]
[1 0 1 0 0 1 1 0]
[1 0 0 0 1 0 0 0]
[1 1 0 0 0 1 0 0]
[0 1 1 1 1 0 1 1]
[0 0 0 0 0 1 1 0]
[1 0 1 1 0 1 0 0]
g_I [1 1 1 0 0 0 0 0]
[1 1 1 0 0 1 1 1]
[1 0 1 0 0 0 0 1]
[0 0 1 1 1 1 0 0]
[1 1 0 0 1 0 0 0]
[1 1 1 0 1 1 1 1]
[0 0 1 1 1 0 0 0]
[0 0 0 1 0 0 0 1]
g_I [0 1 1 1 0 1 0 0]
[1 0 1 0 0 1 1 0]
[0 1 1 1 0 